# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pacific grove,36.6177,-121.9166,12.46,85,0,2.24,US,2023-02-08 02:29:56
1,1,jamestown,42.0970,-79.2353,2.80,93,100,5.66,US,2023-02-08 02:29:56
2,2,avarua,-21.2078,-159.7750,29.03,79,28,5.14,CK,2023-02-08 02:29:57
3,3,kurumkan,54.3000,110.3000,-18.83,89,98,1.19,RU,2023-02-08 02:29:57
4,4,beringovskiy,63.0500,179.3167,-27.30,94,82,3.45,RU,2023-02-08 02:29:57


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_humidity=city_data_df.hvplot.points(
    "Lng", 
    "Lat",
    geo=True,
    tiles ="OSM",
    frame_width = 800,
    frame_height = 500,
    size="Humidity",
    color = "City"
)

# Display the map
map_plot_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
my_ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] <=30) & (city_data_df["Humidity"]<=50)]
my_ideal_weather
# Drop any rows with null values

my_ideal_weather = my_ideal_weather.dropna()


# Display sample data
my_ideal_weather.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,mentougou,39.9392,116.0917,2.52,40,2,0.82,CN,2023-02-08 02:29:59
21,21,bama,11.5221,13.6856,19.13,46,39,2.63,NG,2023-02-08 02:30:23
31,31,busselton,-33.6500,115.3333,26.10,44,0,0.72,AU,2023-02-08 02:30:38
53,53,winslow,35.0242,-110.6974,2.10,38,0,3.60,US,2023-02-08 02:30:47
73,73,geraldton,-28.7667,114.6000,29.23,45,0,3.09,AU,2023-02-08 02:26:47


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_ideal_weather[['City','Country','Lat','Lng','Humidity']].copy()
#hotel_df.head()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel_Name"]= ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel_Name
10,mentougou,CN,39.9392,116.0917,40,
21,bama,NG,11.5221,13.6856,46,
31,busselton,AU,-33.6500,115.3333,44,
53,winslow,US,35.0242,-110.6974,38,
73,geraldton,AU,-28.7667,114.6000,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000 

params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': '北工院会议中心', 'country': 'China', 'country_code': 'cn', 'state': 'Beijing', 'city': 'Shijingshan District', 'town': 'Wulitun', 'hamlet': '敬德寺', 'postcode': '100042', 'street': '石门路', 'lon': 116.12065965453564, 'lat': 39.955255050000005, 'formatted': '北工院会议中心, 石门路, 100042 Beijing, China', 'address_line1': '北工院会议中心', 'address_line2': '石门路, 100042 Beijing, China', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': '北工院会议中心', 'osm_id': 1081678619, 'tourism': 'hotel', 'building': 'yes', 'osm_type': 'w'}}, 'distance': 3035, 'place_id': '51c62f2d4aba075d40595ae7f00442fa4340f00102f9011b1b794000000000920315e58c97e5b7a5e999a2e4bc9ae8aeaee4b8ade

{'type': 'FeatureCollection', 'features': []}
bama - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Broadwater Beach Resort', 'country': 'Australia', 'country_code': 'au', 'state': 'Western Australia', 'city': 'City Of Busselton', 'suburb': 'Broadwater', 'street': 'Holgate Road', 'lon': 115.2797776748616, 'lat': -33.6595186, 'state_code': 'WA', 'formatted': 'Broadwater Beach Resort, Holgate Road, Broadwater WA, Australia', 'address_line1': 'Broadwater Beach Resort', 'address_line2': 'Holgate Road, Broadwater WA, Australia', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation', 'internet_access'], 'details': ['details.accommodation', 'details.facilities'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Broadwater Beach Resort', 'star

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Ocean Centre Hotel', 'country': 'Australia', 'country_code': 'au', 'state': 'Western Australia', 'city': 'Geraldton', 'municipality': 'City Of Greater Geraldton', 'postcode': '6530', 'district': 'Geraldton', 'street': 'Foreshore Drive', 'lon': 114.6082072169853, 'lat': -28.773845700000003, 'state_code': 'WA', 'formatted': 'Ocean Centre Hotel, Foreshore Drive, Geraldton WA 6530, Australia', 'address_line1': 'Ocean Centre Hotel', 'address_line2': 'Foreshore Drive, Geraldton WA 6530, Australia', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': ['details', 'details.accommodation', 'details.contact'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Ocean Centre Hotel', 'rooms': 92, 'stars': 4, 'osm_id': 4

{'type': 'FeatureCollection', 'features': []}
shakiso - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': []}
yatou - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Woodbridge on the Derwent', 'country': 'Australia', 'country_code': 'au', 'state': 'Tasmania', 'city': 'New Norfolk', 'municipality': 'Derwent Valley', 'street': 'Lyell Highway', 'lon': 147.0575608, 'lat': -42.7795192, 'state_code': 'TAS', 'formatted': 'Woodbridge on the Derwent, Lyell Highway, New Norfolk TAS, Australia', 'address_line1': 'Woodbridge on the Derwent', 'address_line2': 'Lyell Highway, New Norfolk TAS, Australia', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Woodbridge on the Derwent', 'osm_id': 25427

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Hotel Sagar and Restaurant', 'country': 'India', 'country_code': 'in', 'state': 'Rajasthan', 'state_district': 'Rajsamand District', 'county': 'Rajsamand Tehsil', 'city': 'Badarda', 'hamlet': 'Dabela', 'postcode': '313300', 'street': 'NH58', 'lon': 73.84189666398447, 'lat': 24.9907575, 'state_code': 'RJ', 'formatted': 'Hotel Sagar and Restaurant, NH58, Badarda - 313300, Rajasthan, India', 'address_line1': 'Hotel Sagar and Restaurant', 'address_line2': 'NH58, Badarda - 313300, Rajasthan, India', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Hotel Sagar and Restaurant', 'osm_id': 524656248, 'tourism': 'hotel', 'building': 'yes', 'osm_type': 'w'}}, 

...

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'country': 'Argentina', 'country_code': 'ar', 'state': 'Chubut Province', 'county': 'Departamento Escalante', 'city': 'Comodoro Rivadavia', 'postcode': 'U9000', 'district': 'La Loma', 'suburb': 'Centro', 'street': 'Leandro Alem', 'lon': -67.4990326, 'lat': -45.8706586, 'formatted': 'Leandro Alem, Centro, U9000 Comodoro Rivadavia, Argentina', 'address_line1': 'Leandro Alem', 'address_line2': 'Centro, U9000 Comodoro Rivadavia, Argentina', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'osm_id': 2202695467, 'tourism': 'hotel', 'osm_type': 'n'}}, 'distance': 446, 'place_id': '51d3286e26f0df50c0598df2b2bd71ef46c0f00103f9012b774a8300000000'}, 'geometry': {'type': 'Point', 'coordinates': [-67.4990326, -45.870

{'type': 'FeatureCollection', 'features': []}
bosobolo - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Prestige Deluxe', 'country': 'India', 'country_code': 'in', 'state_district': 'Ratnagiri', 'county': 'Guhagar', 'city': 'Guhagar', 'postcode': '415703', 'street': 'Chiplun Guhagar Road', 'lon': 73.1914371, 'lat': 17.4843635, 'formatted': 'Prestige Deluxe, Chiplun Guhagar Road, Guhagar - 415703, India', 'address_line1': 'Prestige Deluxe', 'address_line2': 'Chiplun Guhagar Road, Guhagar - 415703, India', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Prestige Deluxe', 'osm_id': 2389650329, 'tourism': 'hotel', 'osm_type': 'n', 'designation': 'http://www.prestigedeluxelodge.com/'}}, 'distance': 156, 'pl

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'country': 'Democratic Republic of the Congo', 'country_code': 'cd', 'state': 'Tshopo', 'city': 'Basoko', 'street': 'RP405', 'lon': 23.6047281, 'lat': 1.2298781, 'state_code': 'TO', 'formatted': 'RP405, Basoko, Democratic Republic of the Congo', 'address_line1': 'RP405', 'address_line2': 'Basoko, Democratic Republic of the Congo', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'osm_id': 5260061146, 'tourism': 'hotel', 'osm_type': 'n'}}, 'distance': 1617, 'place_id': '51e278f475cf9a374059571c90a894adf33ff00103f901da29863901000000'}, 'geometry': {'type': 'Point', 'coordinates': [23.604728099999996, 1.229878099869685]}}]}
basoko - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': []}

{'type': 'FeatureCollection', 'features': []}
coihueco - nearest hotel: No hotel found
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Терней', 'country': 'Russia', 'country_code': 'ru', 'region': 'Far Eastern Federal District', 'state': 'Primorsky Krai', 'county': 'Terneysky District', 'city': 'Terney', 'postcode': '692150', 'street': 'Комсомольская улица', 'housenumber': '13', 'lon': 136.6132738, 'lat': 45.0495873, 'formatted': 'Терней, Комсомольская улица 13, Terney, Primorsky Krai, Russia, 692150', 'address_line1': 'Терней', 'address_line2': 'Комсомольская улица 13, Terney, Primorsky Krai, Russia, 692150', 'categories': ['accommodation', 'accommodation.hotel'], 'details': ['details'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Терней', 'osm_id': 4339653465, 'tourism': 'hotel', 'osm_type': 'n',

,City,Country,Lat,Lng,Humidity,Hotel_Name,Hotel Name
10,mentougou,CN,39.9392,116.0917,40,,北工院会议中心
21,bama,NG,11.5221,13.6856,46,,No hotel found
31,busselton,AU,-33.6500,115.3333,44,,Broadwater Beach Resort
53,winslow,US,35.0242,-110.6974,38,,La Posada Hotel
73,geraldton,AU,-28.7667,114.6000,45,,Ocean Centre Hotel
...,...,...,...,...,...,...,...
565,clarence town,AU,-32.5833,151.7833,38,,Erringhi Hotel
572,khetia,IN,21.6667,74.5833,26,,No hotel found
573,djibo,BF,14.1022,-1.6306,16,,No hotel found
580,lisala,CD,2.1500,21.5167,31,,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

map_plot_hotels=hotel_df.hvplot.points(
    "Lng", 
    "Lat",
    geo=True,
    tiles ="OSM",
    frame_width = 800,
    frame_height = 500,
    size="Humidity",
    color = "City"
)


# Display the map
map_plot_hotels

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)